In [1]:
import lsdb

In [2]:
cat = lsdb.open_catalog("/Users/smcmu/code/lsdb/tests/data/small_sky_order3_source")
cat

,source_id,source_ra,source_dec,mjd,mag,band,object_id,object_ra,object_dec
npartitions=43,,,,,,,,,
"Order: 3, Pixel: 264",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 707",...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 755",...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 760",...,...,...,...,...,...,...,...,...


In [3]:
cat.write_catalog("/tmp/resume_test", overwrite=True)

Writing partitions:   0%|          | 0/86 [00:00<?, ?it/s]

In [6]:
import random

def random_error(df):
    if random.random() < 0.1:
        raise ValueError("Random error")
    return df
cat.map_partitions(random_error).write_catalog("/tmp/resume_test_err", overwrite=True)

Writing partitions:   0%|          | 0/86 [00:00<?, ?it/s]

RuntimeError: Error applying function random_error to partition 40: Random error

In [7]:
!tree /tmp/resume_test_err

/tmp/resume_test_err
├── collection.properties
└── small_sky_order3_source
    └── dataset
        └── Norder=3
            └── Dir=0
                ├── Npix=710.parquet
                ├── Npix=711.parquet
                ├── Npix=714.parquet
                ├── Npix=719.parquet
                ├── Npix=720.parquet
                ├── Npix=741.parquet
                ├── Npix=745.parquet
                ├── Npix=755.parquet
                └── Npix=760.parquet

5 directories, 10 files


In [8]:
cat.write_catalog("/tmp/resume_test_err", resume=True)

Writing partitions:   0%|          | 0/68 [00:00<?, ?it/s]

In [9]:
cat.compute()

Computing Catalog:   0%|          | 0/44 [00:00<?, ?it/s]

,source_id,source_ra,source_dec,mjd,mag,band,object_id,object_ra,object_dec
_healpix_29,,,,,,,,,
1188997068234845449,72125,348.077158,-29.339061,58508.200151,15.090111,u,779,347.5,-29.5
1189091599648562723,82225,347.994565,-29.261689,59219.758815,18.127542,r,779,347.5,-29.5
...,...,...,...,...,...,...,...,...,...
3424289397526654168,70604,307.553185,-25.007915,58406.843975,16.936253,u,743,307.5,-25.5
3424301471875806021,86081,307.606224,-24.926633,59488.818985,16.745674,i,743,307.5,-25.5
